<a href="https://colab.research.google.com/github/afortuny/DeepLearningFastAI/blob/main/13_language_model_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 719 kB 31.8 MB/s 
     |████████████████████████████████| 1.3 MB 65.1 MB/s 
     |████████████████████████████████| 365 kB 51.7 MB/s 
     |████████████████████████████████| 4.7 MB 84.2 MB/s 
     |████████████████████████████████| 212 kB 100.4 MB/s 
     |████████████████████████████████| 115 kB 92.0 MB/s 
     |████████████████████████████████| 120 kB 81.6 MB/s 
     |████████████████████████████████| 127 kB 95.0 MB/s 
     |████████████████████████████████| 6.6 MB 50.2 MB/s 
Mounted at /content/gdrive


In [2]:
#hide
from fastbook import *
from IPython.display import display,HTML

# Music model from scratch

In [3]:
!pip install music21
!apt-get install -y lilypond

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-droid-fallback fonts-lmodern fonts-noto-mono ghostscript gsfonts
  libauthen-sasl-perl libcupsfilters1 libcupsimage2 libdata-dump-perl
  libencode-locale-perl libfile-listing-perl libfont-afm-perl libgs9
  libgs9-common libhtml-form-perl libhtml-format-perl libhtml-parser-perl
  libhtml-tagset-perl libhtml-tree-perl libhttp-cookies-perl
  libhttp-daemon-perl libhttp-date-perl libhttp-message-perl
  libhttp-negotiate-perl libijs-0.35 libio-html-perl libio-socket-ssl-perl
  libjbig2dec0 libkpathsea6 liblwp-mediatypes-perl liblwp-protocol-https-perl
  libmailtools-perl libnet-http-perl libnet-smtp-ssl-per

In [4]:
#Importing Libraries
import numpy as np 
import pandas as pd 
from collections import Counter
import random
import IPython
from IPython.display import Image, Audio
import music21
from music21 import *
import matplotlib.pyplot as plt 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import sys
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
np.random.seed(42)

In [5]:
#Loading the list of chopin's midi files as stream 
filepath = "/content/gdrive/MyDrive/Music/chopin"
#Getting midi files
all_midis= []
for i in os.listdir(filepath):
    if i.endswith(".mid"):
        tr = filepath+'/'+i
        midi = converter.parse(tr)
        all_midis.append(midi)

In [6]:
all_midis

[<music21.stream.Score 0x7f75ae6573d0>,
 <music21.stream.Score 0x7f75adcb2d50>,
 <music21.stream.Score 0x7f75ae09a290>,
 <music21.stream.Score 0x7f75adc07790>,
 <music21.stream.Score 0x7f75ab126950>,
 <music21.stream.Score 0x7f75ac790c50>,
 <music21.stream.Score 0x7f75acc22b90>,
 <music21.stream.Score 0x7f75a96161d0>,
 <music21.stream.Score 0x7f75a661d1d0>,
 <music21.stream.Score 0x7f75a8104d10>,
 <music21.stream.Score 0x7f75a8055050>,
 <music21.stream.Score 0x7f75a928e8d0>,
 <music21.stream.Score 0x7f75ab18a990>,
 <music21.stream.Score 0x7f75a805cf50>,
 <music21.stream.Score 0x7f75a5fd4950>,
 <music21.stream.Score 0x7f75a5901950>,
 <music21.stream.Score 0x7f75a5d08790>,
 <music21.stream.Score 0x7f75a5407ad0>,
 <music21.stream.Score 0x7f75a448b190>,
 <music21.stream.Score 0x7f75a422b510>,
 <music21.stream.Score 0x7f75a5770410>,
 <music21.stream.Score 0x7f75a330c590>,
 <music21.stream.Score 0x7f75a3422410>,
 <music21.stream.Score 0x7f75a5e26390>,
 <music21.stream.Score 0x7f75ad30b7d0>,


In [7]:
#Helping function        
def extract_notes(file):
    notes = []
    pick = None
    for j in file:
        songs = instrument.partitionByInstrument(j)
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))

    return notes
#Getting the list of notes as Corpus
Corpus= extract_notes(all_midis)
print("Total notes in all the Chopin midis in the dataset:", len(Corpus))

Total notes in all the Chopin midis in the dataset: 63429


In [8]:
Corpus

['B3',
 '4.8',
 'E2',
 'B2',
 '4.8',
 'B3',
 '3.6',
 'B1',
 'B2',
 '3.6',
 'B3',
 'B3',
 '4.8',
 'E2',
 'B2',
 '4.8',
 'C#4',
 '4.9',
 'A2',
 'C#3',
 '4.9',
 'G#2',
 'C#4',
 '6.9',
 'F#2',
 'C#3',
 '6.9',
 'E-4',
 '6.9',
 'C#3',
 '6.9',
 'E-4',
 'E-4',
 '6.11',
 'B1',
 'E-3',
 '6.11',
 'E-4',
 '8.11',
 'B2',
 'E-3',
 '8.11',
 'B2',
 'E4',
 '8.11',
 'C#2',
 'E3',
 '8.11',
 'E4',
 '1.6',
 'A2',
 'E3',
 '1.6',
 'E4',
 'A2',
 'E4',
 '6.11',
 'B1',
 'E3',
 '6.11',
 'E-4',
 '6.11',
 '10.11',
 '11.1',
 '11.1',
 '11.1',
 'E-3',
 '11.1',
 '11.1',
 '11.1',
 '6.11',
 '11.1',
 '11.1',
 'E-4',
 '11.1',
 'E-4',
 '8.11',
 '10.11',
 'C#2',
 'E3',
 '8.11',
 'E-4',
 '6.10',
 'F#2',
 'E3',
 '6.10',
 'C#4',
 'F#2',
 'C#4',
 '6.9',
 'E-2',
 'C#3',
 '6.9',
 'C#4',
 '6.9',
 '2.3',
 '3.4',
 '3.4',
 '3.4',
 'B2',
 '3.4',
 '3.4',
 '3.4',
 '6.9',
 '3.4',
 '3.4',
 'B3',
 '1.3',
 'B3',
 '4.8',
 '4.6',
 'E2',
 'B2',
 '4.8',
 'B3',
 '3.6',
 'B1',
 'B2',
 '3.6',
 'B3',
 'B3',
 '2.7',
 'G1',
 'B2',
 '2.7',
 'B3',
 '2.

In [9]:
from fastai.text.all import *

In [10]:
text = ' . '.join([l.strip() for l in Corpus])
text[:100]

'B3 . 4.8 . E2 . B2 . 4.8 . B3 . 3.6 . B1 . B2 . 3.6 . B3 . B3 . 4.8 . E2 . B2 . 4.8 . C#4 . 4.9 . A2'

In [11]:
text = text.replace('.','')

In [12]:
text

'B3  48  E2  B2  48  B3  36  B1  B2  36  B3  B3  48  E2  B2  48  C#4  49  A2  C#3  49  G#2  C#4  69  F#2  C#3  69  E-4  69  C#3  69  E-4  E-4  611  B1  E-3  611  E-4  811  B2  E-3  811  B2  E4  811  C#2  E3  811  E4  16  A2  E3  16  E4  A2  E4  611  B1  E3  611  E-4  611  1011  111  111  111  E-3  111  111  111  611  111  111  E-4  111  E-4  811  1011  C#2  E3  811  E-4  610  F#2  E3  610  C#4  F#2  C#4  69  E-2  C#3  69  C#4  69  23  34  34  34  B2  34  34  34  69  34  34  B3  13  B3  48  46  E2  B2  48  B3  36  B1  B2  36  B3  B3  27  G1  B2  27  B3  27  G2  B2  27  B3  F2  C4  G3  C3  G3  D4  710  D2  D3  710  E4  710  C2  E3  710  E4  70  0  E3  70  E4  11  E4  91  9  E3  91  E4  101  7  E3  101  E4  7  F4  101  5  F3  101  G4  101  4  G3  101  G#4  80  3  E-3  E-4  80  E-4  70  E-3  710  C#4  C4  G#3  G#1  E-3  G#3  B3  B3  38  B1  B2  39  B3  B3  48  4  B2  48  B3  36  B1  B2  36  B3  B3  48  E2  B2  48  C4  49  9  C3  49  9  C4  59  5  C3  59  C4  47  0  C3  47  C4  C4  59  5  C

In [13]:
tokens = text.split(' ')
tokens[:10]

['B3', '', '48', '', 'E2', '', 'B2', '', '48', '']

In [14]:
vocab = L(*tokens).unique()

In [15]:
len(vocab)

318

In [16]:
seql = 15

In [17]:
test_list =  tokens
odd_i = []
even_i = []
for i in range(0, len(test_list)):
    if i % 2:
        even_i.append(test_list[i])
    else :
        odd_i.append(test_list[i])
 
tokens = odd_i

In [18]:
word2idx = {w:i for i,w in enumerate(vocab)}
nums = L(word2idx[i] for i in tokens)
nums

(#63429) [0,2,3,4,2,0,5,6,4,5...]

In [19]:
L((tokens[i:i+3], tokens[i+3]) for i in range(0,len(tokens)-4,3))

(#21142) [(['B3', '48', 'E2'], 'B2'),(['B2', '48', 'B3'], '36'),(['36', 'B1', 'B2'], '36'),(['36', 'B3', 'B3'], '48'),(['48', 'E2', 'B2'], '48'),(['48', 'C#4', '49'], 'A2'),(['A2', 'C#3', '49'], 'G#2'),(['G#2', 'C#4', '69'], 'F#2'),(['F#2', 'C#3', '69'], 'E-4'),(['E-4', '69', 'C#3'], '69')...]

In [20]:
seqs = L((tensor(nums[i:i+3]), nums[i+3]) for i in range(0,len(nums)-4,3))
seqs

(#21142) [(tensor([0, 2, 3]), 4),(tensor([4, 2, 0]), 5),(tensor([5, 6, 4]), 5),(tensor([5, 0, 0]), 2),(tensor([2, 3, 4]), 2),(tensor([2, 7, 8]), 9),(tensor([ 9, 10,  8]), 11),(tensor([11,  7, 12]), 13),(tensor([13, 10, 12]), 14),(tensor([14, 12, 10]), 12)...]

In [21]:
bs = 64
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(seqs[:cut], seqs[cut:], bs=64, shuffle=False)

In [22]:
class LMModel2(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        
    def forward(self, x):
        h = 0
        for i in range(3):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
        return self.h_o(h)

In [23]:
learn = Learner(dls, LMModel2(len(vocab), 32), loss_func=F.cross_entropy, 
                metrics=accuracy)
learn.fit_one_cycle(4, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.975929,4.881926,0.017735,00:01
1,4.527922,4.794359,0.022464,00:01
2,4.452067,4.755588,0.031922,00:01
3,4.401902,4.726520,0.034287,00:01


In [24]:
class LMModel3(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
        
    def forward(self, x):
        for i in range(3):
            self.h = self.h + self.i_h(x[:,i])
            self.h = F.relu(self.h_h(self.h))
        out = self.h_o(self.h)
        self.h = self.h.detach()
        return out
    
    def reset(self): self.h = 0

In [25]:
m = len(seqs)//bs
m,bs,len(seqs)

(330, 64, 21142)

In [26]:
def group_chunks(ds, bs):
    m = len(ds) // bs
    new_ds = L()
    for i in range(m): new_ds += L(ds[i + m*j] for j in range(bs))
    return new_ds

In [27]:
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(
    group_chunks(seqs[:cut], bs), 
    group_chunks(seqs[cut:], bs), 
    bs=bs, drop_last=True, shuffle=False)

In [28]:
learn = Learner(dls, LMModel3(len(vocab), 64), loss_func=F.cross_entropy,
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(10, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.770848,4.772535,0.031960,00:01
1,4.333669,4.658344,0.039062,00:01
2,3.932618,4.616055,0.058475,00:01
3,3.663089,4.645497,0.063684,00:01
4,3.477239,4.717246,0.059659,00:01
5,3.331177,4.813365,0.061790,00:01
6,3.223081,4.862075,0.056818,00:01
7,3.127280,4.919221,0.059422,00:01
8,3.062252,4.947471,0.062973,00:01
9,3.020456,4.942518,0.061553,00:01


In [29]:
sl = 15
seqs = L((tensor(nums[i:i+sl]), tensor(nums[i+1:i+sl+1]))
         for i in range(0,len(nums)-sl-1,sl))
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(group_chunks(seqs[:cut], bs),
                             group_chunks(seqs[cut:], bs),
                             bs=bs, drop_last=True, shuffle=False)

In [30]:
class LMModel4(Module):
    def __init__(self, vocab_sz, n_hidden):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)  
        self.h_h = nn.Linear(n_hidden, n_hidden)     
        self.h_o = nn.Linear(n_hidden,vocab_sz)
        self.h = 0
        
    def forward(self, x):
        outs = []
        for i in range(sl):
            self.h = self.h + self.i_h(x[:,i])
            self.h = F.relu(self.h_h(self.h))
            outs.append(self.h_o(self.h))
        self.h = self.h.detach()
        return torch.stack(outs, dim=1)
    
    def reset(self): self.h = 0

In [31]:
def loss_func(inp, targ):
    return F.cross_entropy(inp.view(-1, len(vocab)), targ.view(-1))

In [32]:
learn = Learner(dls, LMModel4(len(vocab), 64), loss_func=loss_func,
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.648573,5.494866,0.016186,00:00
1,5.061223,4.813563,0.016506,00:00
2,4.666489,4.733281,0.030609,00:00
3,4.331367,4.573633,0.053285,00:00
4,4.061513,4.482505,0.061939,00:00
5,3.871891,4.468775,0.062821,00:00
6,3.751327,4.461823,0.070513,00:00
7,3.645895,4.467423,0.066346,00:00
8,3.566453,4.418778,0.074119,00:00
9,3.510100,4.393136,0.075160,00:00


In [33]:
class LMModel5(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.RNN(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = torch.zeros(n_layers, bs, n_hidden)
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(res)
    
    def reset(self): self.h.zero_()

In [34]:
learn = Learner(dls, LMModel5(len(vocab), 64, 2), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.662906,5.415090,0.009776,00:01
1,5.025662,4.832181,0.007372,00:00
2,4.733210,4.622195,0.033894,00:00
3,4.398068,4.455278,0.057051,00:00
4,4.107425,4.377475,0.066827,00:00
5,3.881881,4.341073,0.068029,00:00
6,3.714909,4.312668,0.076603,00:00
7,3.586802,4.303141,0.078686,00:01
8,3.480973,4.306376,0.082853,00:00
9,3.398811,4.320427,0.084375,00:00


In [35]:
class LMModel6(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(2)]
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(res)
    
    def reset(self): 
        for h in self.h: h.zero_()

In [36]:
learn = Learner(dls, LMModel6(len(vocab), 64, 2), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.286301,4.877585,0.008814,00:01
1,4.813663,4.715585,0.018429,00:01
2,4.480261,4.496366,0.044311,00:01
3,4.162977,4.319636,0.068429,00:01
4,3.887309,4.249105,0.078846,00:01
5,3.644440,4.238892,0.083574,00:01
6,3.425519,4.261651,0.082612,00:01
7,3.233510,4.295185,0.079647,00:01
8,3.065640,4.328409,0.077484,00:01
9,2.918372,4.401388,0.071234,00:01


In [37]:
class Dropout(Module):
    def __init__(self, p): self.p = p
    def forward(self, x):
        if not self.training: return x
        mask = x.new(*x.shape).bernoulli_(1-p)
        return x * mask.div_(1-p)

In [38]:
class LMModel7(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers, p):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first=True)
        self.drop = nn.Dropout(p)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h_o.weight = self.i_h.weight
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(2)]
        
    def forward(self, x):
        raw,h = self.rnn(self.i_h(x), self.h)
        out = self.drop(raw)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(out),raw,out
    
    def reset(self): 
        for h in self.h: h.zero_()

In [39]:
learn = TextLearner(dls, LMModel7(len(vocab), 64, 2, 0.4),
                    loss_func=CrossEntropyLossFlat(), metrics=accuracy)

In [40]:
learn.fit_one_cycle(15, 1e-2, wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,5.492869,5.141015,0.048157,00:01
1,4.940295,4.508177,0.072997,00:01
2,4.463783,4.310714,0.082051,00:01
3,4.099197,4.224761,0.081731,00:01
4,3.811024,4.182595,0.092628,00:01
5,3.585671,4.172690,0.093269,00:01
6,3.401755,4.170134,0.095112,00:01
7,3.250787,4.199159,0.098317,00:01
8,3.122117,4.216109,0.093510,00:01
9,3.015888,4.228951,0.099199,00:01
